In [ ]:
####################################################################################################################################################
####################################################################################################################################################
####                                                                                                                                            ####
####                              This pre-trained model, MobileNetV2, was explored because the save size is smaller than                       ####
####                              a TensorFlow Convolutional Network. A smaller sized model will run faster on the Streamlit                    ####
####                              application. I would love to explore additional pretrained models including the Xception model                ####
####                              as it has very high accuracy scores on multi-class image classification problems.                             ####
####                                                                                                                                            ####
####                              Check this paper out: https://arxiv.org/abs/1704.04861                                                        ####
####                                                                                                                                            ####
####################################################################################################################################################
####################################################################################################################################################

# Imports

In [27]:
import tensorflow as tf
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet import preprocess_input
from sklearn.metrics import plot_confusion_matrix, recall_score, accuracy_score, precision_score 
import seaborn as sns
import os
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

# MobileNetV2 Model

#### Download the pre-trained MobileNetV2 model, set the first layers to non-trainable, the final layers to trainable, and add in a few dense layers to learn the PlantVillage dataset.

In [31]:
base_model=tf.keras.applications.MobileNetV2(weights='imagenet',
                                             include_top=False,
                                             input_shape=(224,224,3)) #The required image shape is 224,224,3 isntead of 250,250,3 ran in the Tensor Flow CNN model

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #add dense layers to build on top of the pre-trained MobileNetV2 model 
x=Dense(1024,activation='relu')(x) 
x=Dense(512,activation='relu')(x) 
preds=Dense(37,activation='softmax')(x) 

In [32]:
model=Model(inputs=base_model.input,outputs=preds)
#specify the inputs
#specify the outputs
#now a model has been created based on our architecture

In [33]:
#MobileNetV2 has a much more complex topography than the one that I constructed in 
for i,layer in enumerate(model.layers):
    print(i,layer.name)

0 input_6
1 Conv1
2 bn_Conv1
3 Conv1_relu
4 expanded_conv_depthwise
5 expanded_conv_depthwise_BN
6 expanded_conv_depthwise_relu
7 expanded_conv_project
8 expanded_conv_project_BN
9 block_1_expand
10 block_1_expand_BN
11 block_1_expand_relu
12 block_1_pad
13 block_1_depthwise
14 block_1_depthwise_BN
15 block_1_depthwise_relu
16 block_1_project
17 block_1_project_BN
18 block_2_expand
19 block_2_expand_BN
20 block_2_expand_relu
21 block_2_depthwise
22 block_2_depthwise_BN
23 block_2_depthwise_relu
24 block_2_project
25 block_2_project_BN
26 block_2_add
27 block_3_expand
28 block_3_expand_BN
29 block_3_expand_relu
30 block_3_pad
31 block_3_depthwise
32 block_3_depthwise_BN
33 block_3_depthwise_relu
34 block_3_project
35 block_3_project_BN
36 block_4_expand
37 block_4_expand_BN
38 block_4_expand_relu
39 block_4_depthwise
40 block_4_depthwise_BN
41 block_4_depthwise_relu
42 block_4_project
43 block_4_project_BN
44 block_4_add
45 block_5_expand
46 block_5_expand_BN
47 block_5_expand_relu
48 b

In [34]:
for layer in model.layers[:20]:   #Set some of the layers to be trainable, but most as untrainable to retain the pre-trained aspect of MobileNetV2
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True

#### Pull in data

In [35]:
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator=train_datagen.flow_from_directory('./dataset/train',
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 69079 images belonging to 37 classes.


#### Compile model

In [ ]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

#### Fit model

In [43]:
step_size_train=train_generator.n//train_generator.batch_size
model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   epochs=4)

Epoch 1/4
2158/2158 [==============================] - 9269s 4s/step - loss: 0.2135 - accuracy: 0.9437
Epoch 2/4
2158/2158 [==============================] - 9169s 4s/step - loss: 0.1624 - accuracy: 0.9581
Epoch 3/4
2158/2158 [==============================] - 7860s 4s/step - loss: 0.1334 - accuracy: 0.9646
Epoch 4/4
2158/2158 [==============================] - 7584s 4s/step - loss: 0.1201 - accuracy: 0.9678


#### Save model

In [44]:
model.save('./models/leaf-model00.h5')

In [45]:
model.save('./models/my_model007')

INFO:tensorflow:Assets written to: ./models/my_model007\assets


#### Model Topology

In [90]:
# View Final Model Topology
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_6[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
____________________________________________________________________________________________